In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import os
import numpy as np
import mne
from mne_bids import make_bids_basename

mne.set_log_level('WARNING')

In [2]:
sSub, session, task, run = '08', '01', 'picturenaming', '01'
bids_basename = make_bids_basename(subject=sSub, session=session,
                                  task=task, run=run)
data_dir = '/Users/chholak/data/pic-name-data-bids/'
data_path = data_dir + 'MEG/'
bids_path = data_path + 'sub-' + sSub + '/ses-' + session + '/meg/'

raw_fname = bids_path + bids_basename + '_meg.fif'
epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')

subjects_dir = os.path.join(data_dir, 'MRI/')
subject = 'sub-' + sSub

In [6]:
trans_fname = os.path.join(bids_path, subject + '-trans.fif')
trans_fname

'/Users/chholak/data/pic-name-data-bids/MEG/sub-08/ses-01/meg/sub-08-trans.fif'

In [7]:
trans_fname = os.path.join(bids_path, subject + '-trans.fif')
bem_fname = os.path.join(bids_path, subject + '-bem.fif')
fname_surf_lh = os.path.join(subjects_dir, subject, 'surf', 'lh.white')

In [8]:
bem = mne.bem.read_bem_solution(bem_fname)
bem

<ConductorModel  |  BEM (1 layer)>

In [9]:
info = mne.io.read_info(epochs_fname)
fig = mne.viz.plot_alignment(info, trans_fname, subject=subject, dig=True,
                            subjects_dir=subjects_dir, bem=bem, verbose=True);

Could not find the surface for head in the provided BEM model, looking in the subject directory.
Using outer_skin.surf for head surface.
Getting helmet for system 306m
Using mayavi 3d backend.



In [36]:
epochs = mne.read_epochs(epochs_fname)

In [37]:
epochs.apply_proj()

<EpochsFIF  |   98 events (all good), -0.5 - 1 sec, baseline [-0.5, 0], ~351.3 MB, data loaded,
 'picture': 98>

In [38]:
epochs.pick_types(meg=True, eeg=False)

<EpochsFIF  |   98 events (all good), -0.5 - 1 sec, baseline [-0.5, 0], ~349.0 MB, data loaded,
 'picture': 98>

In [39]:
cov = mne.compute_covariance(epochs, rank='info')

In [40]:
evoked = epochs.average()

In [41]:
evoked.plot(spatial_colors=True, gfp=True);

In [42]:
evoked_crop = evoked.copy().crop(0.13, 0.15)
dip, residual = mne.fit_dipole(evoked_crop, cov, bem_fname, trans_fname)

print(dip)

<Dipole  |  n_times : 21, tmin : 0.130, tmax : 0.150>


In [43]:
print(dip.gof)

[38.41608525 37.99503135 37.21918422 36.11700352 34.74175989 33.16057089
 31.46213994 29.74880574 27.36795561 27.21824179 26.9306975  26.53454267
 26.05738481 25.51977669 24.93696069 25.14492358 26.16572976 27.42347192
 28.81815609 30.24125471 31.58416715]


In [44]:
dip.plot_locations(subject=subject, trans=trans_fname,
                  subjects_dir=subjects_dir, mode='orthoview');

In [47]:
selection = mne.read_selection('Right', info=evoked.info)
dip, residual = \
    mne.fit_dipole(evoked_crop.copy().pick_channels(selection),
                  cov, bem_fname, trans_fname)
print(dip)

<ipython-input-47-ba37277b8543>:4: RuntimeWarning: Projection vector "EOG-planar--0.200-0.200-PCA-01" has magnitude 0.76 (should be unity), applying projector with 102/204 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  cov, bem_fname, trans_fname)
<ipython-input-47-ba37277b8543>:4: RuntimeWarning: Projection vector "EOG-axial--0.200-0.200-PCA-01" has magnitude 0.76 (should be unity), applying projector with 51/102 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  cov, bem_fname, trans_fname)
<ipython-input-47-ba37277b8543>:4: RuntimeWarning: Projection vector "EOG-planar--0.200-0.200-PCA-01" has magnitude 0.76 (should be unity), applying projector with 102/204 of the original 

<Dipole  |  n_times : 21, tmin : 0.130, tmax : 0.150>


In [48]:
print(dip.gof)

[17.14895808 16.39604153 15.58516644 13.75703841 14.24385918 15.03324236
 16.1421105  17.56992892 20.04377441 24.38982403  3.17428685  3.35612474
 33.19994497 35.41539317  3.57006499 38.22273397 38.69962139 38.510042
 36.5100991  34.78553507 32.58515733]


In [50]:
dip.plot_locations(subject=subject, trans=trans_fname,
                  subjects_dir=subjects_dir, mode='orthoview');